In [1]:
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
trainset = sklearn.datasets.load_files(container_path = '/home/yash/Downloads/to_push/NLP-Models-master/Text-Classification/data/', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [17]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
print ONEHOT
print len(trainset.data)
print np.arange(len(trainset.data))
#print trainset.target

ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
print ONEHOT
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, trainset.target, ONEHOT, test_size = 0.2)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
10662
[    0     1     2 ... 10659 10660 10661]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [18]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20465
('Most common words', [(u'the', 10129), (u'a', 7312), (u'and', 6199), (u'of', 6063), (u'to', 4233), (u'is', 3378)])
('Sample data', [4, 662, 9, 2543, 8, 22, 4, 3558, 18064, 98], [u'the', u'rock', u'is', u'destined', u'to', u'be', u'the', u'21st', u'centurys', u'new'])


In [42]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']
print GO, PAD,EOS ,UNK

0 1 2 3


In [30]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_size,
                 dict_size, dimension_output, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.BasicRNNCell(size_layer,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        
        outputs, _ = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype = tf.float32)
        
        W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.contrib.layers.xavier_initializer())
        #W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
        
        self.logits = tf.matmul(outputs[:, -1], W) + b
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.logits, labels = self.Y))
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [39]:
size_layer = 128
num_layers = 4
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-6
maxlen = 50
batch_size = 128

In [40]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

In [41]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 500, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    #print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.513672
epoch: 0, training loss: 0.779013, training acc: 0.491122, valid loss: 0.759074, valid acc: 0.513672

epoch: 1, training loss: 0.773794, training acc: 0.493016, valid loss: 0.755771, valid acc: 0.513184

epoch: 2, pass acc: 0.513672, current acc: 0.515137
epoch: 2, training loss: 0.769165, training acc: 0.494673, valid loss: 0.752857, valid acc: 0.515137

epoch: 3, training loss: 0.764989, training acc: 0.497159, valid loss: 0.750269, valid acc: 0.515137

epoch: 4, training loss: 0.761201, training acc: 0.497396, valid loss: 0.747956, valid acc: 0.515137

epoch: 5, pass acc: 0.515137, current acc: 0.520508
epoch: 5, training loss: 0.757745, training acc: 0.497988, valid loss: 0.745874, valid acc: 0.520508

epoch: 6, training loss: 0.754575, training acc: 0.498935, valid loss: 0.743988, valid acc: 0.519531

epoch: 7, training loss: 0.751650, training acc: 0.500000, valid loss: 0.742268, valid acc: 0.517578

epoch: 8, training loss: 0.7

epoch: 79, training loss: 0.684664, training acc: 0.555516, valid loss: 0.714002, valid acc: 0.499023

epoch: 80, training loss: 0.684272, training acc: 0.556226, valid loss: 0.713980, valid acc: 0.499023

epoch: 81, training loss: 0.683884, training acc: 0.556108, valid loss: 0.713961, valid acc: 0.499023

epoch: 82, training loss: 0.683500, training acc: 0.556937, valid loss: 0.713945, valid acc: 0.499023

epoch: 83, training loss: 0.683119, training acc: 0.556463, valid loss: 0.713931, valid acc: 0.499512

epoch: 84, training loss: 0.682742, training acc: 0.556108, valid loss: 0.713919, valid acc: 0.499512

epoch: 85, training loss: 0.682368, training acc: 0.557410, valid loss: 0.713910, valid acc: 0.499512

epoch: 86, training loss: 0.681998, training acc: 0.558120, valid loss: 0.713903, valid acc: 0.499512

epoch: 87, training loss: 0.681631, training acc: 0.558949, valid loss: 0.713899, valid acc: 0.500977

epoch: 88, training loss: 0.681266, training acc: 0.560251, valid loss: 0

epoch: 158, training loss: 0.659881, training acc: 0.601918, valid loss: 0.717224, valid acc: 0.509766

epoch: 159, training loss: 0.659605, training acc: 0.602273, valid loss: 0.717309, valid acc: 0.508301

epoch: 160, training loss: 0.659330, training acc: 0.602154, valid loss: 0.717395, valid acc: 0.508301

epoch: 161, training loss: 0.659055, training acc: 0.602273, valid loss: 0.717482, valid acc: 0.508789

epoch: 162, training loss: 0.658780, training acc: 0.602865, valid loss: 0.717569, valid acc: 0.508789

epoch: 163, training loss: 0.658506, training acc: 0.603220, valid loss: 0.717658, valid acc: 0.508301

epoch: 164, training loss: 0.658232, training acc: 0.604048, valid loss: 0.717747, valid acc: 0.509277

epoch: 165, training loss: 0.657958, training acc: 0.605114, valid loss: 0.717838, valid acc: 0.509277

epoch: 166, training loss: 0.657685, training acc: 0.605824, valid loss: 0.717929, valid acc: 0.509277

epoch: 167, training loss: 0.657411, training acc: 0.606179, val

KeyboardInterrupt: 

In [ ]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))